## Examine time series for individual patients and quantify how much variance exists from which to make predictions.

Upgrade libraries.

In [3]:
# !pip install dask --upgrade
# !pip install dask[complete]
# !pip install 'fsspec>=0.3.3'
# !pip install s3fs --upgrade
# !pip install msgpack==0.5.6

Install to keep track of long running cell's progress

In [5]:
# !pip install tqdm

Import packages.

In [6]:
import boto3
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client
from tqdm import tqdm
import pandas as pd
from sagemaker import get_execution_role
from d_types import dtypes
import s3fs


In [7]:
client = Client()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [8]:
client

Client Scheduler: tcp://127.0.0.1:44433 Dashboard: http://127.0.0.1:36537/status,Cluster Workers: 4 Cores: 4 Memory: 16.82 GB


In [11]:
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)

Import data to dask dataframe.

In [12]:
role = get_execution_role()

bucket='vbhdata'
data_key = 'VBH_hospdata.txt'
data_location = 's3://{}/{}'.format(bucket, data_key)


In [13]:
df = dd.read_csv(data_location,
                 sep='|',
#                  parse_dates=['_Date'],
                 dtype=dtypes,
                 blocksize=64000000,
                 low_memory=False)

In [15]:
df.head()

,_Date,Clientid,ClientStatus,IsMale,DOB,PrimaryProgramName,MilitaryStatus,Age,HispanicOrigin,Race,Ethnicity,MaritalStatus,CurrentlyHomeless,LivingArrangement,ClientState,Unnamed: 15,ClientCounty,PrimaryLanguage,EducationLevel,EducationStatus,FinanciallyResponsible,EmploymentStatus,EmploymentInformation,ClientMonthlyIncome,ClientAnnualIncome,HouseholdAnnualIncome,PrimarySource,SmokingStatus,AgeOfFirstTobaccoUse,HeadOfHousehold,NumberInHousehold,NumberofDependents,ForensicTreatment,JusticeSystemInvolvement,Admissions,Crisis,SelfHarm,HarmToOthers,HarmToProperty,Pgm_A & D Adult Outpatient,Pgm_A & D Youth Day Tx,Pgm_A.T.I.- J.D.O.T.,Pgm_A.T.I.-C.O.R.E. Outpatient,Pgm_A.T.I.-C.O.R.E. Residential,Pgm_Acute Chldrns Extended Sv,Pgm_Adult Autism Center of Lifetime Learning,Pgm_Adult Centralized Eval,Pgm_Alliance House Apts,Pgm_Assertive Outreach Team,Pgm_Beacon Heights Elem Sch,Pgm_C.O.R.E.2 Outpatient,Pgm_C.O.R.E.2 Residential,Pgm_Carmen B Pingree Elementary School,Pgm_Carmen B Pingree In Home,Pgm_Carmen B Pingree Preschool,Pgm_Centralized Intake Clinic,Pgm_Childrens Outpatient Srvc,Pgm_Childrens Outpatient West,Pgm_Community Based Prov Ntwk,Pgm_Contract-Granite School District,Pgm_Contract-Jordan School District,Pgm_D.B.T. Day Tx Program,Pgm_Federal Forensics Grant,Pgm_Flexcare New Choices,Pgm_Forensic Unit,Pgm_Fresh Start/Recovery Prog,Pgm_H.S.S.C Am Fork,Pgm_H.S.S.C Highland,Pgm_H.S.S.C Riverton,Pgm_H.S.S.C. Boise,Pgm_H.S.S.C. Lab Services,Pgm_H.S.S.C. Layton,Pgm_H.S.S.C. Orem,Pgm_Homefront,Pgm_Independent Living,Pgm_Inpatient Hospital Summit Adult,Pgm_Inpatient Hospital Tooele Adult,Pgm_Inpatient Hospital Tooele Youth,Pgm_K.I.D.S,Pgm_Lab Services,Pgm_Masters Program,Pgm_Medical Records,Pgm_No Primary program,Pgm_North Valley Adult Pgm,Pgm_Pheasant Hollow,Pgm_Residential Tooele Youth,Pgm_Robert Frost Elem Sch,Pgm_S.R.S.,Pgm_Safe Haven 1,Pgm_Safe Haven 2,Pgm_School Based Program,Pgm_Summit County A & D,Pgm_Summit County Mental Hlth,Pgm_Summit Jail,Pgm_Summit JRI Jail,Pgm_Summit JRI Outpt,Pgm_Summit Prevention,Pgm_Summit School Contractor,Pgm_TMS,Pgm_Tooele After School Program,Pgm_Tooele County A & D,Pgm_Tooele County M. H.,Pgm_Tooele County Prevention,Pgm_Tooele Food Bank,Pgm_Tooele I-Wrap,Pgm_Tooele Jail,Pgm_Tooele JRI Jail,Pgm_Tooele JRI Outpt,Pgm_Tooele Resource Center,Pgm_Tooele Wendover,Pgm_Tooele Youth Services,Pgm_USH Summit Youth,Pgm_USH Tooele Adult,Pgm_Valley EPIC Bldg A,Pgm_Valley EPIC Bldg B,Pgm_Valley EPIC Bldg C,Pgm_Valley EPIC Bldg D,Pgm_Valley EPIC Outpatient Recovery Mngmnt,Pgm_Valley Plaza,Pgm_Valley Storefront,Pgm_Valley Woods,Pgm_ValleyAIM,Pgm_ValleyPhoenix,Pgm_ValleyWest,PosTest_6-Acetylmorphine,PosTest_7-Aminoclonazepam,PosTest_7-Hydroxyquetiapine,PosTest_9-hydroxyrisperidone,PosTest_Albumin,PosTest_Alk Phos,PosTest_Alprazolam,PosTest_ALT,PosTest_Amphetamine,PosTest_Amphetamines,PosTest_Anion Gap,PosTest_aOH-Alprazolam,PosTest_Aripiprazole,PosTest_AST,PosTest_Barbiturates,PosTest_Basophil#,PosTest_Basophil%,PosTest_Benzodiazepines,PosTest_Benzoylecgonine,PosTest_BUN,PosTest_Buprenorphine,PosTest_Bupropion,PosTest_Calcium,PosTest_Carisoprodol,PosTest_Chloride,PosTest_Cholesterol,PosTest_Clozapine,PosTest_Codeine,PosTest_Desmethylolanzapine,PosTest_Diazyme Potassium,PosTest_Diazyme Sodium,PosTest_EDDP,PosTest_EGFR (African American),PosTest_EGFR (Non-African American),PosTest_Eosinophil#,PosTest_Eosinophil%,PosTest_EtG,PosTest_Ethyl Alcohol (mg/dL),PosTest_EtS,PosTest_Fentanyl,PosTest_Gabapentin,PosTest_Glucose,PosTest_Haloperidol,PosTest_HbA1c,PosTest_HCT,PosTest_HDL,PosTest_HGB,PosTest_Hydrocodone,PosTest_Hydromorphone,PosTest_Hydroxybupropion,PosTest_LDL Direct,PosTest_LDL/HDL Ratio,PosTest_Lorazepam,PosTest_Lymphocyte#,PosTest_Lymphocyte%,PosTest_MCH,PosTest_MCHC,PosTest_MCV,PosTest_MDA,PosTest_MDEA,PosTest_MDMA,PosTest_Meprobamate,PosTest_Methadone,PosTest_Methamphetamine,PosTest_Mitragynine,PosTest_Monocyte#,PosTest_Monocyte%,PosTest_Morphine,PosTest_MPV,PosTest_N-Desmethyclozapine,PosTest_Neutrophil#,PosTe

Get only what we need to isolate patients who have been hospitalized.

In [17]:
df1 = df[['_Date', 'Clientid', 'Admissions']]
df1 = df1.dropna()

In [18]:
df1.head()

,_Date,Clientid,Admissions
0,2019-01-01,940.0,0.0
1,2019-01-01,1060.0,0.0
2,2019-01-01,1250.0,0.0
3,2019-01-01,1831.0,0.0
4,2019-01-01,2541.0,0.0


In [19]:
df1 = client.persist(df1)
df1 = df1.compute()

Check for null values

In [20]:
df2 = df1.isna().sum()

In [21]:
df2

_Date         0
Clientid      0
Admissions    0
dtype: int64

Filter for hopsitalized patients.

In [24]:
hosp_df = df1[df1['Admissions'] == 1.0]
hosp_df = hosp_df.reset_index(drop=True)
hosp_df.head()

,_Date,Clientid,Admissions
0,2019-01-01,365810.0,1.0
1,2019-01-01,2106790.0,1.0
2,2019-01-01,2107915.0,1.0
3,2019-01-01,2127622.0,1.0
4,2019-01-01,2128389.0,1.0


Get list of clientid's of hospitalized pts and filter whole dataset for them.

In [26]:
hosp_clients = hosp_df['Clientid'].unique()
hosp_clients_df = df[df['Clientid'].isin(hosp_clients)]
hosp_clients_df = client.persist(hosp_clients_df)
hosp_clients_df = hosp_clients_df.compute()
hosp_clients_df.head()

,_Date,Clientid,ClientStatus,IsMale,DOB,PrimaryProgramName,MilitaryStatus,Age,HispanicOrigin,Race,Ethnicity,MaritalStatus,CurrentlyHomeless,LivingArrangement,ClientState,Unnamed: 15,ClientCounty,PrimaryLanguage,EducationLevel,EducationStatus,FinanciallyResponsible,EmploymentStatus,EmploymentInformation,ClientMonthlyIncome,ClientAnnualIncome,HouseholdAnnualIncome,PrimarySource,SmokingStatus,AgeOfFirstTobaccoUse,HeadOfHousehold,NumberInHousehold,NumberofDependents,ForensicTreatment,JusticeSystemInvolvement,Admissions,Crisis,SelfHarm,HarmToOthers,HarmToProperty,Pgm_A & D Adult Outpatient,Pgm_A & D Youth Day Tx,Pgm_A.T.I.- J.D.O.T.,Pgm_A.T.I.-C.O.R.E. Outpatient,Pgm_A.T.I.-C.O.R.E. Residential,Pgm_Acute Chldrns Extended Sv,Pgm_Adult Autism Center of Lifetime Learning,Pgm_Adult Centralized Eval,Pgm_Alliance House Apts,Pgm_Assertive Outreach Team,Pgm_Beacon Heights Elem Sch,Pgm_C.O.R.E.2 Outpatient,Pgm_C.O.R.E.2 Residential,Pgm_Carmen B Pingree Elementary School,Pgm_Carmen B Pingree In Home,Pgm_Carmen B Pingree Preschool,Pgm_Centralized Intake Clinic,Pgm_Childrens Outpatient Srvc,Pgm_Childrens Outpatient West,Pgm_Community Based Prov Ntwk,Pgm_Contract-Granite School District,Pgm_Contract-Jordan School District,Pgm_D.B.T. Day Tx Program,Pgm_Federal Forensics Grant,Pgm_Flexcare New Choices,Pgm_Forensic Unit,Pgm_Fresh Start/Recovery Prog,Pgm_H.S.S.C Am Fork,Pgm_H.S.S.C Highland,Pgm_H.S.S.C Riverton,Pgm_H.S.S.C. Boise,Pgm_H.S.S.C. Lab Services,Pgm_H.S.S.C. Layton,Pgm_H.S.S.C. Orem,Pgm_Homefront,Pgm_Independent Living,Pgm_Inpatient Hospital Summit Adult,Pgm_Inpatient Hospital Tooele Adult,Pgm_Inpatient Hospital Tooele Youth,Pgm_K.I.D.S,Pgm_Lab Services,Pgm_Masters Program,Pgm_Medical Records,Pgm_No Primary program,Pgm_North Valley Adult Pgm,Pgm_Pheasant Hollow,Pgm_Residential Tooele Youth,Pgm_Robert Frost Elem Sch,Pgm_S.R.S.,Pgm_Safe Haven 1,Pgm_Safe Haven 2,Pgm_School Based Program,Pgm_Summit County A & D,Pgm_Summit County Mental Hlth,Pgm_Summit Jail,Pgm_Summit JRI Jail,Pgm_Summit JRI Outpt,Pgm_Summit Prevention,Pgm_Summit School Contractor,Pgm_TMS,Pgm_Tooele After School Program,Pgm_Tooele County A & D,Pgm_Tooele County M. H.,Pgm_Tooele County Prevention,Pgm_Tooele Food Bank,Pgm_Tooele I-Wrap,Pgm_Tooele Jail,Pgm_Tooele JRI Jail,Pgm_Tooele JRI Outpt,Pgm_Tooele Resource Center,Pgm_Tooele Wendover,Pgm_Tooele Youth Services,Pgm_USH Summit Youth,Pgm_USH Tooele Adult,Pgm_Valley EPIC Bldg A,Pgm_Valley EPIC Bldg B,Pgm_Valley EPIC Bldg C,Pgm_Valley EPIC Bldg D,Pgm_Valley EPIC Outpatient Recovery Mngmnt,Pgm_Valley Plaza,Pgm_Valley Storefront,Pgm_Valley Woods,Pgm_ValleyAIM,Pgm_ValleyPhoenix,Pgm_ValleyWest,PosTest_6-Acetylmorphine,PosTest_7-Aminoclonazepam,PosTest_7-Hydroxyquetiapine,PosTest_9-hydroxyrisperidone,PosTest_Albumin,PosTest_Alk Phos,PosTest_Alprazolam,PosTest_ALT,PosTest_Amphetamine,PosTest_Amphetamines,PosTest_Anion Gap,PosTest_aOH-Alprazolam,PosTest_Aripiprazole,PosTest_AST,PosTest_Barbiturates,PosTest_Basophil#,PosTest_Basophil%,PosTest_Benzodiazepines,PosTest_Benzoylecgonine,PosTest_BUN,PosTest_Buprenorphine,PosTest_Bupropion,PosTest_Calcium,PosTest_Carisoprodol,PosTest_Chloride,PosTest_Cholesterol,PosTest_Clozapine,PosTest_Codeine,PosTest_Desmethylolanzapine,PosTest_Diazyme Potassium,PosTest_Diazyme Sodium,PosTest_EDDP,PosTest_EGFR (African American),PosTest_EGFR (Non-African American),PosTest_Eosinophil#,PosTest_Eosinophil%,PosTest_EtG,PosTest_Ethyl Alcohol (mg/dL),PosTest_EtS,PosTest_Fentanyl,PosTest_Gabapentin,PosTest_Glucose,PosTest_Haloperidol,PosTest_HbA1c,PosTest_HCT,PosTest_HDL,PosTest_HGB,PosTest_Hydrocodone,PosTest_Hydromorphone,PosTest_Hydroxybupropion,PosTest_LDL Direct,PosTest_LDL/HDL Ratio,PosTest_Lorazepam,PosTest_Lymphocyte#,PosTest_Lymphocyte%,PosTest_MCH,PosTest_MCHC,PosTest_MCV,PosTest_MDA,PosTest_MDEA,PosTest_MDMA,PosTest_Meprobamate,PosTest_Methadone,PosTest_Methamphetamine,PosTest_Mitragynine,PosTest_Monocyte#,PosTest_Monocyte%,PosTest_Morphine,PosTest_MPV,PosTest_N-Desmethyclozapine,PosTest_Neutrophil#,PosTe

In [27]:
hosp_clients_df.shape

(366393, 238)

Store dataframe for each client in a dict.

In [28]:
hosp_clients_dict = {clientid: hosp_clients_df[hosp_clients_df['Clientid'] == clientid].set_index('_Date')
                     for clientid in tqdm(hosp_clients.tolist())}

100%|██████████| 1642/1642 [00:08<00:00, 190.54it/s]


In [81]:
def total_changes(df):
    """Total all changes in given client dataframe."""
    sums = []
    length = len(df)
    for column in df.columns:
        # Skip Admissions column, we need changes that may predict this.
        if column == 'Admissions':
            continue
        total = df[column].value_counts()
        if not total.empty:
            # Check that there are more than one value in the entire column.
            sums.append(total.values[0] != length)
    # Sum of all columns with multiple values.
    return sum(sums)

Test it out on a few client's dataframes.

In [82]:
total_changes(hosp_clients_dict[hosp_clients[1]])

0

In [83]:
total_changes(hosp_clients_dict[hosp_clients[2]])

0

In [84]:
total_changes(hosp_clients_dict[hosp_clients[3]])

0

Get total number of columns containing more than one value for all clients.

In [88]:
changes = [total_changes(hosp_clients_dict[client]) for client in tqdm(hosp_clients)]
sum(changes)

100%|██████████| 1642/1642 [07:52<00:00,  3.47it/s]


1284

Total columns with multiple values per client.

In [92]:
sum(changes) / len(changes)

0.781973203410475

This does not bode well for making predictions. There is very little variance